### Import Basic Libraries

In [50]:
import unicodedata
import tensorflow as tf
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os 
import io
import time

import string

import xml.etree.ElementTree as ET

### Import Dataset

In [51]:
tree = ET.parse('C://Users//rochitranjan//Documents//NEWS2018_DATASET_04//NEWS2018_DATASET_04//NEWS2018_M-EnHi_trn.xml')

elemList = []

for elem in tree.iter():
    elemList.append(elem.tag)

root = tree.getroot()

trans = {}
for ele in root.findall('Name'):
    w = ele.find('SourceName').text
    w = w.strip(string.punctuation)
    w = w.strip().split(' ')
    tgt = ele.find('TargetName').text.split(' ')
    if len(w) == len(tgt):
        for ix in range(len(w)):
            trans[w[ix]] = tgt[ix] 

### Preprocessing the input and output sequence

In [52]:
def preprocess_sentence(w):
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = list(w)
  w = ['<sos>'] + w + ['<eos>']
  return w

input_sentences = [list(keys) for keys in trans.keys()]
output_sentences = [list(values) for values in trans.values()]

ip_sent = []
for ip in input_sentences:
    ip_sent.append(['<sos>'] + ip + ['<eos>']) #append the start and end tags to the tokenised sentences
                                                          #each tokenied sentence is stored as a list in output sentences


op_sent = []
for op in output_sentences:
    op_sent.append(['<sos>'] + op + ['<eos>']) #append the start and end tags to the tokenised sentences
                                                          #each tokenied sentence is stored as a list in output sentences


In [53]:
# Function to split words into characters and convert words into character id sequences.
# Additionally, this function applies padding to the end of the sequence to standardise the sequence length
def tokenize(lang, pad): 
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')

  lang_tokenizer.fit_on_texts(lang)
  
  tensor = lang_tokenizer.texts_to_sequences(lang)
  
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

In [54]:
# function takes the list of input and target sequence and 
# returns the input tensor and output tensor for all the input and output sequence
def load_dataset(inp_lang, targ_lang):
  # creating cleaned input, output pairs
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang, 'post')
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang, 'post')

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [55]:
# Calling the load_dataset function to get input and target sequence in tensor form
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(ip_sent, op_sent)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2,random_state=7)

max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [56]:
# BUFFER_SIZE stores the number of training points
BUFFER_SIZE = len(input_tensor_train)

# BATCH_SIZE is set to 64. Training and gradient descent happens in batches of 32
BATCH_SIZE = 32

# the number of batches in one epoch (also, the number of steps during training, when we go batch by batch)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE

# the length of the embedded vector
embedding_dim = 128

# no of GRUs
units = 512 

# getting the size of the input and output vocabularies.
vocab_inp_size = len(inp_lang.word_index)+1 
vocab_tar_size = len(targ_lang.word_index)+1

# now, we shuffle the dataset and split it into batches of 32
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # the remainder after splitting by 32 are dropped

In [57]:
# to understand the shape of an input batch
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([32, 24]), TensorShape([32, 24]))

### Encoder - Decoder using Attention Model

In [58]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz # set batch size
    self.enc_units = enc_units # set the number of GRU units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # set the embedding layer using the input's vocabulary size and the embedding dimension (which is set to 256). This learnt before feeding
                                                                          # as an input to the GRU.

    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True, # Return output of the GRU
                                   return_state=True, # Return hidden state of the GRU
                                   recurrent_initializer='glorot_uniform') # define the GRU layer

  def call(self, x, hidden): # this function is invoked when the function encoder is called with an input and an initialised hidden layer
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden) # pass input x into the GRU layer and initialise the hidden state using initialize_hidden_state 
    return output, state # function returns the encoder output and the hidden state. In a traditional architecture of Seq2Seq modelling, output of Encoder is not used
                         # rather hidden state of the encoder output is feed into decoder


  def initialize_hidden_state(self): #intialise hidden layer to all zeroes (for determining the shape)
    return tf.zeros((self.batch_sz, self.enc_units))

In [59]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE) # create an Encoder class object

# sample input to get a sense of the shapes.
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (32, 24, 512)
Encoder Hidden state shape: (batch size, units) (32, 512)


`The Below class gives an implementation of Bahdanau Additive Attention layer`

In [60]:
# a class defined for the attention layer
# returns attention weights and context vector.

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units) # fully-connected dense layer-1
    self.W2 = tf.keras.layers.Dense(units) # fully-connected dense layer-2
    self.V = tf.keras.layers.Dense(1) # fully-connected dense layer-3

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [61]:
attention_layer = BahdanauAttention(20) # create an attention layer object
attention_result, attention_weights = attention_layer(sample_hidden, sample_output) # pass sample encoder output and hidden layer to get a sense of the shape of the output of the attention layer.

print("Attention result shape (context vector): (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape (context vector): (batch size, units) (32, 512)
Attention weights shape: (batch_size, sequence_length, 1) (32, 24, 1)


In [62]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz # batch_size which is defined as 32
    self.dec_units = dec_units # the number of decoder GRU units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # defining an embedding layer for the target language output. 
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform') # GRU layer
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output) # getting the context vector and the attention weights from the attention layer

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x) # creating an embedding layer for the target output

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
 
    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))
    

    # output shape == (batch_size, vocab)
    x = self.fc(output) # pass the output through the dense layer

    return x, state, attention_weights # return decoder output, decoder state and attention weights

In [63]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (32, 84)


In [64]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none') #Loss function is categorical crossentropy which is the loss function used for typical classification problem.
                                        # The reason we used from_logits = True is because the output of the decoder given by FC layer is in the form of logits

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0)) # the mask is used to ensure that the zero paddings which we have used is not using in modelling/training.
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [65]:
checkpoint_dir = './tutorial_checkpoint_nmt'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

### Training and Evaluation of the Attention Model

In [66]:
@tf.function
def train_step(inp, targ, enc_hidden):
# the train_step function defines one step of the training process. So, we have to repeat it for all the training items.
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden) # encoder has a inp(input) which is tensor for all of the elements of the
                                                      # input language

    dec_hidden = enc_hidden # decoder hidden value is the encoder_hidden value

    dec_input = tf.expand_dims([targ_lang.word_index['<sos>']] * BATCH_SIZE, 1) # the first value of the decoder input is set as the <SOS> string

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_hidden and enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output) # In case of a decoder we give only one value out of the tensor batch_size and
                                                               # expect only one output from decoder which is different from encoder
                                                               # because entire tensor batch_size is feed into the encoder all at once.

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables # encoder.trainable_variables = weights of encoder
                                                                        # decorder.trainable_variables = weights of decoder

  gradients = tape.gradient(loss, variables) 

  optimizer.apply_gradients(zip(gradients, variables)) # doing gradient descent

  return batch_loss

In [97]:
train = True
EPOCHS = 30
if train :
  for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    # for each of the data point in the batch_size we put it thru the train_step
      batch_loss = train_step(inp, targ, enc_hidden)
      total_loss += batch_loss

      if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                    batch,
                                                    batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.0514
Epoch 1 Batch 100 Loss 0.0491
Epoch 1 Batch 200 Loss 0.2025
Epoch 1 Batch 300 Loss 0.0798
Epoch 1 Loss 0.0899
Time taken for 1 epoch 169.95730328559875 sec

Epoch 2 Batch 0 Loss 0.0430
Epoch 2 Batch 100 Loss 0.0602
Epoch 2 Batch 200 Loss 0.0512
Epoch 2 Batch 300 Loss 0.0415
Epoch 2 Loss 0.0598
Time taken for 1 epoch 169.66071248054504 sec

Epoch 3 Batch 0 Loss 0.0359
Epoch 3 Batch 100 Loss 0.0600
Epoch 3 Batch 200 Loss 0.0498
Epoch 3 Batch 300 Loss 0.0660
Epoch 3 Loss 0.0462
Time taken for 1 epoch 168.73134779930115 sec

Epoch 4 Batch 0 Loss 0.0341
Epoch 4 Batch 100 Loss 0.0315
Epoch 4 Batch 200 Loss 0.0439
Epoch 4 Batch 300 Loss 0.0259
Epoch 4 Loss 0.0376
Time taken for 1 epoch 168.80465507507324 sec

Epoch 5 Batch 0 Loss 0.0253
Epoch 5 Batch 100 Loss 0.0341
Epoch 5 Batch 200 Loss 0.0343
Epoch 5 Batch 300 Loss 0.0287
Epoch 5 Loss 0.0336
Time taken for 1 epoch 168.33061981201172 sec

Epoch 6 Batch 0 Loss 0.0277
Epoch 6 Batch 100 Loss 0.0234
Epoch 6 Batch 200

In [165]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [166]:
def evaluate(word):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  word = preprocess_sentence(word)

  inputs = [inp_lang.word_index[i] for i in word]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden) # for encoder a zero matrix is provided as initial state for the first time, while initialising it.

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<sos>']], 0) # define <SOS> as the first input to the decoder

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    
    # pass the encoder output, decoder hidden state(which is initialised to encoder hidden state for the first time and decoder input to the decoder)
    # make a prediction and obtain decoder hidden states

    predicted_id = tf.argmax(predictions[0]).numpy() # Use the word_id which has the maximum probability of occurence as the prediction for each of the decoder output

    if targ_lang.index_word[predicted_id] != '<eos>':
      result += targ_lang.index_word[predicted_id] + ' '
    else:
      return result, word#, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, word#, attention_plot

In [167]:
def transliterate(word):
  result, word = evaluate(word)
#  print('Input: %s' % (word))
  print('Predicted transliteration: {}'.format(result.replace(' ', '')))
  return result.replace(' ', '')

### Sample Outputs

In [223]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: algorithm
Predicted transliteration: एल्गोरिथम


'एल्गोरिथम'

In [225]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: abhigyaan
Predicted transliteration: अभिग्ञान


'अभिग्ञान'

In [226]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: namaste
Predicted transliteration: नमस्ते


'नमस्ते'

In [227]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: nonsense
Predicted transliteration: नॉनसेंस


'नॉनसेंस'

In [228]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: idiot
Predicted transliteration: इडियोत


'इडियोत'

In [229]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: important
Predicted transliteration: इम्पोर्टेंत


'इम्पोर्टेंत'

In [230]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: pneumonia
Predicted transliteration: प्नूमोनिया


'प्नूमोनिया'

In [231]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: horse
Predicted transliteration: हॉर्स


'हॉर्स'

In [232]:
word = input('Type your input word: ')
transliterate(list(word))

Type your input word: generous
Predicted transliteration: जनरोउस


'जनरोउस'